# Insérer titre pertinent et aguicheur 

In [ ]:
!pip install pandas
!pip install geopandas

On importe avant tout les librairies intéressantes !

In [3]:
import pandas as pd
import requests
import geopandas as gpd

On définit ensuite une fonction qui permet de récupérer des données à partir d'une API

In [2]:
def get_dataset_from_url(api_url):
    '''Imports a database from an API url.
    Args:
        api_url (string) : url of the wanted API
    Returns:
        pandas.DataFrame : dataframe
    '''
    req=requests.get(api_url)
    wb=req.json()
    df = pd.json_normalize(wb["results"])
    #dataset = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)

    return df

## Téléchargement des données de consommation d'électricité annuelle par adresse

In [4]:
consumption_data_url="https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv?lang=fr&refine=annee%3A%222021%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"

response=requests.get(consumption_data_url)
if response.status_code == 200:
    with open("consommation.csv", "wb") as file:
        file.write(response.content)
    print("Téléchargement réussi.")
else:
    print(f"Échec du téléchargement. Code d'état : {response.status_code}")

Téléchargement réussi.


In [5]:
df = pd.read_csv("consommation.csv", sep=";")
df.head(3)

,Année,Code IRIS,Nom IRIS,Numéro de voie,Indice de répétition,Type de voie,Libellé de voie,Code INSEE de la commune,Nom de la commune,Segment de client,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Tri des adresses,Code EPCI,Code Département,Code Région
0,2021,452840102,Mermoz Clos du Hameau-Vieux Bourg,76.0,B,AVENUE,LOUIS JOSEPH SOULAS,45284,Saint-Jean-de-Braye,RESIDENTIEL,12,25.597,2.133,4.144,76 B AVENUE LOUIS JOSEPH SOULAS,330006,244500468.0,45.0,24.0
1,2021,452840102,Mermoz Clos du Hameau-Vieux Bourg,70.0,B,AVENUE,LOUIS JOSEPH SOULAS,45284,Saint-Jean-de-Braye,RESIDENTIEL,20,50.517,2.526,4.144,70 B AVENUE LOUIS JOSEPH SOULAS,330007,244500468.0,45.0,24.0
2,2021,452840302,Sainte-Marie-Diderot-Richaudière-Frédeville,33.0,NaN,AVENUE,PIERRE ET MARIE CURIE,45284,Saint-Jean-de-Braye,RESIDENTIEL,11,24.591,2.236,4.144,33 AVENUE PIERRE ET MARIE CURIE,330017,244500468.0,45.0,24.0
5,2018,341540105,Carnon,141,None,PLACE,DES ESQUIFS,34154,MAUGUIO,RESIDENTIEL,53,93.192,1.758,4.802,141 PLACE DES ESQUIFS,580029,243400470,34,76
6,2018,341540101,Centre ville,108,None,RUE,DU SAUT DU LOUP,34154,MAUGUIO,RESIDENTIEL,26,117.236,4.509,4.802,108 RUE DU SAUT DU LOUP,580040,243400470,34,76
7,2018,593930000,Maulde (commune non irisée),35,None,RUE,DE CHORETTE,59393,MAULDE,RESIDENTIEL,11,74.983,6.817,5.743,35 RUE DE CHORETTE,580067,200042190,59,32
8,2018,783800102,Quartier Bas,2,None,RUE,D AGNOU,78380,MAULE,RESIDENTIEL,12,55.399,4.617,6.907,2 RUE D AGNOU,580068,200034130,78,11
9,2018,783800102,Quartier Bas,7,None,RUE,DU PRESSOIR,78380,MAULE,RESIDENTIEL,13,79.217,6.094,6.907,7 RUE DU PRESSOIR,580074,200034130,78,11


In [11]:
consumption_2018['nom_iris'].unique()

array(['Pont Allant', 'Pommeraye', 'Fictif - Mauges-sur-Loire', 'Carnon',
       'Quartier ouest', 'Centre ville', 'Maulde (commune non irisée)',
       'Quartier Bas'], dtype=object)

Problème ! La base de données est pour l'instant trop petite !